In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [26]:
base_url = "https://www.atptour.com/en/rankings/singles"
columns = ['Country','Source','Name','Age']
data = pd.DataFrame(columns = columns)

proxies = {
  'http': 'www-proxy.us.oracle.com:80',
  'https': 'www-proxy.us.oracle.com:80',
}

url = base_url 
source_code = requests.get(url,proxies=proxies)
plain_text = source_code.text
soup = BeautifulSoup(plain_text, 'html.parser')
table_body = soup.find('tbody')
for row in table_body.findAll('tr'):
        td = row.findAll('td')
        country = td[2].find('img').get('alt')
        source = td[3].find('a').get('href')
        name = td[3].find('a').text
        age = td[4].text.strip()
        player_data = pd.DataFrame([[country,source,name,age]])
        player_data.columns = columns
        data = data.append(player_data, ignore_index=True)
data = data.drop_duplicates()

# Get detailed player information from player page
detailed_columns = ['Win-Loss', 'Titles', 'Prize Money','Source']
detailed_data = pd.DataFrame(index = range(0, 100), columns = detailed_columns)
detailed_data.Source = data.Source.values
#print(detailed_data)

player_data_url = 'https://www.atptour.com'
for id in data.Source:
    url = player_data_url + str(id)
    source_code = requests.get(url,proxies=proxies)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')
    div_class = soup.find('div', {'class': 'overview-stats-table-wrapper'})
    table_def = div_class.find('table',{'class':'players-stats-table'})
    table_body = table_def.find('tbody')
    row = table_body.findAll('tr')
    td = row[1].findAll('td')
    Win_loss = td[2].find('div').get('data-singles')
    Titles = td[3].find('div').get('data-singles')
    Prize_money = td[4].find('div').get('data-singles')
    
    detailed_data.loc[detailed_data.Source == id, 'Win-Loss'] = Win_loss
    detailed_data.loc[detailed_data.Source == id, 'Titles'] = Titles
    detailed_data.loc[detailed_data.Source == id, 'Prize Money'] = Prize_money
detailed_data = detailed_data.drop_duplicates()
#print(detailed_data.head(5))
full_data = pd.merge(data, detailed_data, how = 'inner', on = 'Source')
print(full_data.head(5))
#full_data.to_csv('data.csv', encoding='utf-8-sig')  
    
        

  Country                                           Source  \
0     SRB         /en/players/novak-djokovic/d643/overview   
1     ESP           /en/players/rafael-nadal/n409/overview   
2     GER       /en/players/alexander-zverev/z355/overview   
3     SUI          /en/players/roger-federer/f324/overview   
4     ARG  /en/players/juan-martin-del-potro/d683/overview   

                    Name Age  Win-Loss Titles   Prize Money  
0         Novak Djokovic  31   846-176     73  $128,804,799  
1           Rafael Nadal  32   925-191     80  $104,754,905  
2       Alexander Zverev  21    184-91     10   $16,177,835  
3          Roger Federer  37  1188-261    100  $121,268,063  
4  Juan Martin del Potro  30   433-170     22   $25,375,273  
